In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = 'azure'
openai.api_base = 'https://nous.openai.azure.com/'
openai.api_version = '2023-07-01-preview'
openai.api_key = ''

In [3]:
engine = 'nous'

In [4]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=0.7,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [11]:
data = []
with open('paragraph-lom.agc.jsonl') as fopen:
    for l in fopen:
        data.append({
            'paragraph': json.loads(l)
        })
        
len(data)

8320

In [12]:
data[0]

{'paragraph': '17 Oktober 2014 \n17 October 2014\n\nP.U. (A) 279   WARTA KERAJAAN PERSEKUTUAN\n\nFEDERAL GOVERNMENT \nGAZETTE   PERINTAH KAWALAN BEKALAN \n(BARANG-BARANG KAWALAN) (NO. 5) 2014\n\n\n\nCONTROL OF SUPPLIES \n(CONTROLLED ARTICLES) (NO. 5) ORDER 2014   DISIARKAN OLEH/ \nPUBLISHED BY\n\nJABATAN PEGUAM NEGARA/ \nATTORNEY GENERAL’S CHAMBERSP.U. (A) 279\n\n2\n\nAKTA KAWALAN BEKALAN 1961\n\nPERINTAH KAWALAN BEKALAN (BARANG-BARANG KAWALAN) (NO. 5) 2014   PADA menjalankan kuasa yang diberikan oleh seksyen 5 Akta Kawalan Bekalan 1961\n\n[Akta 122], Menteri membuat perintah yang berikut:\n\n\n\nNama\n\n1. Perintah ini bolehlah dinamakan Perintah Kawalan Bekalan\n\n(Barang-Barang Kawalan) (No. 5) 2014.\n\n\n\nBarang-barang kawalan\n\n2. Barang-barang yang dinyatakan dalam Jadual diisytiharkan sebagai\n\nbarang-barang kawalan bagi tempoh dari 18 Oktober 2014 hingga 27 Oktober 2014.\n\n\n\nJADUAL\n\nBARANG-BARANG KAWALAN\n\n\n\n1. Daging kambing tempatan bertulang\n\n2. Daging kambing b

In [13]:
!mkdir question-lom-agc

mkdir: cannot create directory ‘question-lom-agc’: File exists


In [14]:
def generate(filename, l):
    if os.path.exists(filename):
        return
    
    s = f"""
{l['paragraph']}

generate a question related to the context
"""
    s = s.strip()
    r = predict(s)
    l['question'] = r

    with open(filename, 'w') as fopen:
        json.dump(l, fopen)

In [15]:
generate('question-lom-agc/0.json', data[0])

In [17]:
from tqdm import tqdm

max_worker = 2
for i in tqdm(range(0, len(data), max_worker)):
    b = data[i: i + max_worker]
    filenames = [(os.path.join('question-lom-agc', f'{i + k}.json'), l) for k, l in enumerate(b)]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, *f): f for f in filenames}

        for future in as_completed(futures):
            future.result()

  5%|██                                    | 227/4160 [05:59<1:32:48,  1.42s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


  6%|██▏                                   | 236/4160 [06:12<1:35:09,  1.46s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 17%|██████▌                               | 723/4160 [19:06<1:27:42,  1.53s/it]

...

 89%|████████████████████████████████▉    | 3697/4160 [1:37:33<11:37,  1.51s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 98%|████████████████████████████████████▎| 4077/4160 [1:47:07<02:14,  1.62s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


100%|█████████████████████████████████████| 4160/4160 [1:49:17<00:00,  1.58s/it]
